# CS 178 Exploration

In [9]:
%pip install kagglehub pandas matplotlib -U --quiet
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.exceptions import ConvergenceWarning
from sklearn.neural_network import MLPClassifier

Note: you may need to restart the kernel to use updated packages.


In [10]:
seed = 178
adult_data = pd.read_csv('data/adult.data', header=None)

## Need to import cols, as they are not pre-defined
adult_data.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']
adult_data = adult_data.replace(' ?', pd.NA) ## Original stored nas as ' ?'

adult_data['income'] = adult_data['income'].apply(lambda x: 1 if x.strip() == '>50K' else 0) # Label Classifier

# Get factors, label vector
X = adult_data.drop(columns=['income'])
y = adult_data['income']
X = pd.get_dummies(X, drop_first=True)
y = pd.get_dummies(y, drop_first=True).iloc[:, 0]

n_features = X.shape[1]

In [11]:
# print(adult_data.describe())

# Convert non-numeric columns to categorical features and print levels and labels
for col in X.select_dtypes(include=['object']).columns:
    X[col] = X[col].astype('category')
    print(f"Column: {col}")
    categories = X[col].cat.categories
    codes = X[col].cat.codes
    sorted_codes = sorted(codes.unique())
    print("Categories and Codes in ascending order by code:")
    for code in sorted_codes:
        print(f"Code: {code}, Category: {categories[code]}")
    print("\n")

for col in X.select_dtypes(include=['number']).columns:
    var = np.var(X[col])
    mean = np.mean(X[col])
    print(f"Column: {col}")
    print(f"Variance: \t {var:.6}\t Mean: \t {mean:.6}")

# Get pi = P(Y = 1)
print(f"Proportion of > 50K Income: {np.mean(y):.6}")

# Compute the F-value and p-value for each feature, toward factor analysis
f_values, p_values = f_classif(X.apply(lambda col: col.cat.codes if col.dtype.name == 'category' else col), y)
f_scores = pd.DataFrame({'Feature': X.columns, 'F-Score': f_values, 'p-Value': p_values})
f_scores = f_scores.sort_values(by=['F-Score', 'p-Value'], ascending=False)

print(f_scores)

Column: age
Variance: 	 186.056	 Mean: 	 38.5816
Column: fnlwgt
Variance: 	 1.11405e+10	 Mean: 	 1.89778e+05
Column: education_num
Variance: 	 6.61869	 Mean: 	 10.0807
Column: capital_gain
Variance: 	 5.45409e+07	 Mean: 	 1077.65
Column: capital_loss
Variance: 	 1.62372e+05	 Mean: 	 87.3038
Column: hours_per_week
Variance: 	 152.454	 Mean: 	 40.4375
Proportion of > 50K Income: 0.24081


                               Feature      F-Score   p-Value
29  marital_status_ Married-civ-spouse  8025.842062  0.000000
2                        education_num  4120.095780  0.000000
31       marital_status_ Never-married  3674.200147  0.000000
0                                  age  1886.707314  0.000000
5                       hours_per_week  1813.386282  0.000000
..                                 ...          ...       ...
60                native_country_ Cuba     0.260277  0.609934
67              native_country_ Greece     0.195063  0.658739
76             native_country_ Ireland     0.138552  0.709727
73             native_country_ Hungary     0.007171  0.932516
89            native_country_ Scotland     0.005546  0.940637

[97 rows x 3 columns]


In [12]:
## Modelling Data Prepping
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

## Logistic Regression Model

In [13]:
## Initialize

try:
    # logit_model = LogisticRegression(max_iter=1) # Accuracy: 0.7608
    logit_model = LogisticRegression(max_iter=2000) # Accuracy: 0.8497
except ConvergenceWarning:
    print('Model Converged')

## Fit
logit_model.fit(X_train, y_train)

feature_names = ['Intercept'] + X_train.columns.tolist()
coefficients = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': [logit_model.intercept_[0]] + logit_model.coef_[0].tolist()
})

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
## Logistic Regression Results
accuracy = logit_model.score(X_test, y_test)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8497


## Simple NN

In [15]:
# Define and initialize
model = MLPClassifier(hidden_layer_sizes=(n_features, n_features), activation='logistic', solver='adam', max_iter=50, random_state=seed)

# Fit
model.fit(X_train, y_train)


MLPClassifier(activation='logistic', hidden_layer_sizes=(97, 97), max_iter=50,
              random_state=178)

In [16]:
# Evaluate
accuracy = model.score(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.7651


White-box approaches look more accurate - This feature set has well labeled and ordinal factors, so this makes sense. Lets try 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=20, random_state=seed) #Random Forest Test Accuracy: 0.8494
rf_model = RandomForestClassifier(n_estimators=100, random_state=seed)

# Train the model
rf_model.fit(X_train, y_train)

# Evaluate the model
rf_accuracy = rf_model.score(X_test, y_test)
print(f"Random Forest Test Accuracy: {rf_accuracy:.4f}")

Random Forest Test Accuracy: 0.8494
